## Imports

In [7]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import json
import jsonschema
from jsonschema.exceptions import ValidationError
import os
from tqdm.notebook  import tqdm
import pathlib
import os.path
import polars as pl 
import xarray as xr
import matplotlib.pyplot as plt
import requests
import seaborn as sns
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
import time
import os
import numpy as np
from IPython.display import display

### **Next Steps** 

1 - Add more values insight

In [5]:
# import suntzu
# import inspect

# # Obtém todos os membros da classe Cleaning
# cleaning_members = inspect.getmembers(suntzu.Cleaning, inspect.isfunction)

# # Itera sobre os membros e imprime suas docstrings
# for member_name, member_obj in cleaning_members:
#     docstring = inspect.getdoc(member_obj)
#     print(f"Function: Cleaning.{member_name}\nDocstring: {docstring}\n")


In [2]:
import suntzu as snt
df = snt.read_file("examples/wdw.nc")
df.capitalize_cols_name()

AttributeError: partially initialized module 'suntzu' has no attribute 'File' (most likely due to a circular import)

In [5]:
if isinstance(df.statistics, snt.Statistics):
    print("A classe Statistics é uma propriedade do objeto df")
else:
    print("A classe Statistics não é uma propriedade do objeto df")

AttributeError: <suntzu.file.File object at 0x7a2688fe6b00> object has no attribute 'statistics'

In [3]:
awd = snt.read_file("examples/updated_Titanic.parquet")
awd.read_parquet_metadata()

Gender
    No attributes were found for this column.
Age
    No attributes were found for this column.
Siblings_on_Board
    No attributes were found for this column.
Parents_on_Board
    No attributes were found for this column.
Ticket_Price
    No attributes were found for this column.
Port_of_Embarkation
    No attributes were found for this column.
Class
    No attributes were found for this column.
Adult/Child
    No attributes were found for this column.
Alone
    No attributes were found for this column.
Survived
    No attributes were found for this column.


In [4]:
teste = snt.read_file("examples/Employee_Cleaned.parquet")
teste.get_best_dtypes(show_df=True)

,Column_Name,Dtype,Best_Dtype
0,EmployeeIdentification,int16,int16
1,Age,int8,int8
2,Attrition,category,category
3,BusinessTravel,category,category
4,DailyRate,int16,int16
5,Department,category,category
6,DistanceFromHome,int8,int8
7,Education,category,category
8,EducationField,category,category
9,EmployeeCount,int16,int16


6 - Visualization Functions

In [41]:
class Visualization:
    def save_telegram_credentials(bot_token=None, chat_id=None):
        """
        Save the Telegram credentials to a JSON file.

        Args:
            bot_token (str, optional): The bot token. If not provided, the user will be prompted to enter it.
            chat_id (str, optional): The chat ID. If not provided, the user will be prompted to enter it.
            filename (str, optional): The name of the JSON file to save the credentials. Defaults to "telegram_credentials.json".
        """
        bot_token = bot_token or input("Insert the bot_token: ")
        chat_id = chat_id or input("Insert the chat id: ")
        dictionary = {"chat_id": chat_id, "bot_token": bot_token}
        try:
            with open('telegram_credentials.json', "w") as outfile:
                json.dump(dictionary, outfile)
        except Exception as e:
            print("Error occurred while saving telegram credentials:", str(e))
    def send_images_via_telegram(file_path, chat_id=None, bot_token=None, caption="This is a caption"):
        """
        Sends an image via Telegram using the provided file path, chat ID, bot token, and caption.

        Args:
            file_path (str): The path to the image file.
            chat_id (str, optional): The ID of the chat to send the image to. If not provided, it will be retrieved from the 'telegram_credentials.json' file. Defaults to None.
            bot_token (str, optional): The token of the Telegram bot. If not provided, it will be retrieved from the 'telegram_credentials.json' file. Defaults to None.
            caption (str, optional): The caption for the image. Defaults to "This is a caption".

        Raises:
            ValueError: If chat_id and bot_token are not provided and the 'telegram_credentials.json' file does not exist.
            ValueError: If chat_id or bot_token is not provided.

        Returns:
            None
        """
        if chat_id is None and bot_token is None:
            if os.path.exists('telegram_credentials.json'):
                try:
                    with open('telegram_credentials.json', 'r') as openfile:
                        json_object = json.load(openfile)
                    chat_id = json_object.get("chat_id")
                    bot_token = json_object.get("bot_token")
                except ValueError:
                    print("Please use the function 'save_telegram_credentials'")
            else:
                raise ValueError("Please provide the chat_id and the bot_token or use the function 'save_telegram_credentials'.")
        if chat_id is None or bot_token is None:
            raise ValueError("chat_id and bot_token are required parameters")
        base_url = f"https://api.telegram.org/bot{bot_token}/sendPhoto"
        with open(file_path, 'rb') as my_file:
            parameters = {
                "chat_id": chat_id,
                "caption": caption
            }
            files = {   
                "photo": my_file
            }
            try:
                resp = requests.post(base_url, data=parameters, files=files)
                status_code = resp.status_code
                if status_code == 200:
                    print("The photo was sent.")
                else:
                    resp_json = resp.json()
                    print("Sent","-", resp_json.get("ok"))
                    del resp_json["ok"]
                    for key, values in resp_json.items():
                        print(key.capitalize(), "-", values)
            except requests.exceptions.RequestException as e:
                print("An error occurred during the request:", str(e))
    def help_telegram_bot():
        """
        Provides information on how to use a Telegram bot.
        """
        print('''
        1. How to create a bot: https://www.directual.com/lesson-library/how-to-create-a-telegram-bot
        2. Adding the bot to a group: https://botifi.me/en/help/telegram-adding-bot-to-channel-or-group/
        3. Getting the bot_token: https://botifi.me/en/help/telegram-existed-bot/
        4. Getting the chat_id of a group: https://www.wikihow.com/Know-Chat-ID-on-Telegram-on-Android
        5. Possible errors: https://core.telegram.org/api/errors
        ''')
    
    def save_slack_credentials(channel_id = None, slack_token=None):
        """
        Saves Slack credentials (channel ID and token) to a JSON file.

        Args:
            channel_id (str, optional): The ID of the Slack channel. If not provided, the user will be prompted to enter it.
            slack_token (str, optional): The Slack token. If not provided, the user will be prompted to enter it.

        Returns:
            None: The function does not return any value.
        """
        slack_token = slack_token or input("Insert the slack_token: ")
        channel_id = channel_id or input("Insert the channel_id: ")
        dictionary = {"channel_id": channel_id, "slack_token": slack_token}
        try:
            with open('slack_credentials.json', "w") as outfile:
                json.dump(dictionary, outfile)
        except Exception as e:
            print("Error occurred while saving slack credentials:", str(e))
    def send_images_via_slack(file_path, channel_id=None, slack_token=None, caption="This is a caption"):
        """
        Sends an image file to a specified Slack channel using the Slack API.

        Args:
            file_path (str): The path to the image file to be sent.
            channel_id (str, optional): The ID of the Slack channel to send the image to. If not provided, it will attempt to read the channel ID from a JSON file named 'slack_credentials.json'.
            slack_token (str, optional): The Slack API token. If not provided, it will attempt to read the token from the same JSON file mentioned above.
            caption (str, optional): The caption to be displayed with the image in Slack.

        Raises:
            ValueError: If 'slack_credentials.json' file is not found or the values are not valid.
            ValueError: If either `channel_id` or `slack_token` is missing.

        Returns:
            None
        """
        if channel_id is None and slack_token is None:
            if os.path.exists('slack_credentials.json'):
                try:
                    with open('slack_credentials.json', 'r') as openfile:
                        json_object = json.load(openfile)
                    channel_id = json_object.get("channel_id")
                    slack_token = json_object.get("slack_token")
                except ValueError:
                    print("Please use the function 'save_slack_credentials'")
            else:
                raise ValueError("Please provide the channel_id and the slack_token or use the function 'save_slack_credentials'.")
        if channel_id is None or slack_token is None:
            raise ValueError("channel_id and slack_token are required parameters")
        client = WebClient(token=slack_token)
        try:
            response = client.files_upload(
                channels=channel_id,
                file=file_path,
                title=caption
            )
            if response["ok"]:
                print("The photo was sent.")
            else:
                for key, value in response.items():
                    print(f"{key.capitalize()}: {value}")
        except SlackApiError as e:
            print(f"Error uploading file: {e.response['error']}")
    def help_slack_bot():
        """
        Provides a list of resources to help users create and configure a Slack bot.
        """
        print('''
            1. Creating a slack_bot (read the first paragraph): https://medium.com/applied-data-science/how-to-build-you-own-slack-bot-714283fd16e5
            2. Getting the channel_id (read method 1): https://www.process.st/how-to/find-slack-channel-id/ 
            ''')

# Testes

In [ ]:
df = pd.read_csv("./Titanic.csv")
df1 = pd.read_csv("./Sales_Data.csv")
df2 = pd.read_parquet("./Titanic_Cleaned.parquet")

In [ ]:
# Define the string
my_string = "HellO, wOrld!"

# Define the character you want to check for
character_to_check = "o"

# Check if the character is in the string
if character_to_check.lower() in my_string.lower():
    print(f"The character '{character_to_check}' is in the string.")
else:
    print(f"The character '{character_to_check}' is not in the string.")


The character 'o' is in the string.


In [ ]:
import pandas as pd

# Creating a DataFrame for testing
data = {
    'name': ['Alice', 'Bob', 'Charlie', 'David', 'Emma'],
    'age': [25, 30, 35, 40, None],
    'Gender': ['female', 'male', None, 'male', 'female'],
    'City': ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix'],
    'SCORE': [85.5, None, 90.7, 75.2, 88.9]
}
df = pd.DataFrame(data)
df.head()

,name,age,Gender,City,SCORE
0,Alice,25.0,female,New York,85.5
1,Bob,30.0,male,Los Angeles,NaN
2,Charlie,35.0,None,Chicago,90.7
3,David,40.0,male,Houston,75.2
4,Emma,NaN,female,Phoenix,88.9


In [ ]:
df.head()

,name,age,Gender,City,SCORE
0,Alice,25.0,female,New York,85.5
1,Bob,30.0,male,Los Angeles,NaN
2,Charlie,35.0,None,Chicago,90.7
3,David,40.0,male,Houston,75.2
4,Emma,NaN,female,Phoenix,88.9


In [ ]:
df1.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,150502.0,iPhone,1.0,700.00,02/18/19 01:35,"866 Spruce St, Portland, ME 04101"
1,150503.0,AA Batteries (4-pack),1.0,3.84,02/13/19 07:24,"18 13th St, San Francisco, CA 94016"
2,150504.0,27in 4K Gaming Monitor,1.0,389.99,02/18/19 09:46,"52 6th St, New York City, NY 10001"
3,150505.0,Lightning Charging Cable,1.0,14.95,02/02/19 16:47,"129 Cherry St, Atlanta, GA 30301"
4,150506.0,AA Batteries (4-pack),2.0,3.84,02/28/19 20:32,"548 Lincoln St, Seattle, WA 98101"


In [ ]:
df_capitalize = Cleaning.split_rows_string(df1, col="Purchase Address", new_cols=["City", "State",], delete_col=False)
df_capitalize.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,City,State
0,150502.0,iPhone,1.0,700.00,02/18/19 01:35,ME 04101,866 Spruce St,"Portland, ME 04101"
1,150503.0,AA Batteries (4-pack),1.0,3.84,02/13/19 07:24,CA 94016,18 13th St,"San Francisco, CA 94016"
2,150504.0,27in 4K Gaming Monitor,1.0,389.99,02/18/19 09:46,NY 10001,52 6th St,"New York City, NY 10001"
3,150505.0,Lightning Charging Cable,1.0,14.95,02/02/19 16:47,GA 30301,129 Cherry St,"Atlanta, GA 30301"
4,150506.0,AA Batteries (4-pack),2.0,3.84,02/28/19 20:32,WA 98101,548 Lincoln St,"Seattle, WA 98101"


In [ ]:
erro

NameError: name 'erro' is not defined

In [ ]:
# self.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   sex       891 non-null    object 
 1   age       714 non-null    float64
 2   sibsp     891 non-null    int64  
 3   parch     891 non-null    int64  
 4   fare      891 non-null    float64
 5   embarked  889 non-null    object 
 6   class     891 non-null    object 
 7   who       891 non-null    object 
 8   alone     891 non-null    bool   
 9   survived  891 non-null    int64  
dtypes: bool(1), float64(2), int64(3), object(4)
memory usage: 63.6+ KB


In [ ]:
# teste = File.read_file("./Titanic_Cleaned.parquet")
# teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   Gender               891 non-null    category
 1   Age                  714 non-null    float64 
 2   Siblings_on_Board    891 non-null    int8    
 3   Parents_on_Board     891 non-null    int8    
 4   Ticket_Price         891 non-null    float64 
 5   Port_of_Embarkation  889 non-null    category
 6   Class                891 non-null    category
 7   Adult/Child          891 non-null    category
 8   Alone                891 non-null    bool    
 9   Survived             891 non-null    int64   
dtypes: bool(1), category(4), float64(2), int64(1), int8(2)
memory usage: 27.6 KB


In [ ]:
import pandas as pd
# Criar um DataFrame de exemplo
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Emma'],
    'Age': [25, 30, 35, 40, 45],
    'Gender': ['Female', 'Male', 'Male', 'Male', 'Female'],
    'City': ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix']
}
df = pd.DataFrame(data)
df.head()

,Name,Age,Gender,City
0,Alice,25,Female,New York
1,Bob,30,Male,Los Angeles
2,Charlie,35,Male,Chicago
3,David,40,Male,Houston
4,Emma,45,Female,Phoenix


In [ ]:
# Conditions for filtering
conditions = [
    df['Age'] <= 35,
    df['Gender'] == 'Male'
]
df_find = Statistics.find(df, conditions)
df_find.head()

,Name,Age,Gender,City
1,Bob,30,Male,Los Angeles
2,Charlie,35,Male,Chicago


In [ ]:
df_replaced = Statistics.find_replace(df, conditions, ('Age', 'Unknown'))
df_replaced.head()

,Name,Age,Gender,City
0,Alice,25,Female,New York
1,Bob,Unknown,Male,Los Angeles
2,Charlie,Unknown,Male,Chicago
3,David,40,Male,Houston
4,Emma,45,Female,Phoenix


In [ ]:
# df_deleted = Statistics.find_delete(df, conditions)
# df_deleted.head()

,Name,Age,Gender,City
0,Alice,25,Female,New York
3,David,40,Male,Houston
4,Emma,45,Female,Phoenix


In [ ]:
# Statistics.get_best_dtypes(self, output=True, convert=True)

The best dtype for sex is category
But consider changing it to bool, has you have 2 unique values so you can map the numbers to be True or False
The best dtype for age is float16
The best dtype for sibsp is int8
The best dtype for parch is int8
The best dtype for fare is float16
The best dtype for embarked is category
The best dtype for class is category
The best dtype for who is category
The best dtype for alone is bool
The best dtype for survived is int8
But consider changing it to bool, has you have 2 unique values so you can map the numbers to be True or False


,sex,age,sibsp,parch,fare,embarked,class,who,alone,survived
0,male,22.0,1,0,7.2500,S,Third,man,False,0
1,female,38.0,1,0,71.2833,C,First,woman,False,1
2,female,26.0,0,0,7.9250,S,Third,woman,True,1
3,female,35.0,1,0,53.1000,S,First,woman,False,1
4,male,35.0,0,0,8.0500,S,Third,man,True,0
...,...,...,...,...,...,...,...,...,...,...
886,male,27.0,0,0,13.0000,S,Second,man,True,0
887,female,19.0,0,0,30.0000,S,First,woman,True,1
888,female,NaN,1,2,23.4500,S,Third,woman,False,0
889,male,26.0,0,0,30.0000,C,First,man,True,1


In [ ]:
# Statistics.get_null_percentage(teste, get_dict=True, output=False, get_total=False)

{'Gender': '0.0%',
 'Age': '19.87%',
 'Siblings_on_Board': '0.0%',
 'Parents_on_Board': '0.0%',
 'Ticket_Price': '0.0%',
 'Port_of_Embarkation': '0.22%',
 'Class': '0.0%',
 'Adult/Child': '0.0%',
 'Alone': '0.0%',
 'Survived': '0.0%'}

In [ ]:
# Metadata.read_netCDF_metadata(teste)

AttributeError: 'DataFrame' object has no attribute 'variables'

In [ ]:
# teste.head()

In [ ]:
# get_nulls_count(teste1)

In [ ]:
# import pandas as pd

# # Creating a sample DataFrame
# data = {
#     'col1': ['apple, pie', 'banana! split', 'cherry? cake'],
#     'col2': ['ice-cream', 'chocolate? cake', 'strawberry! shortcake']
# }

# teste = pd.DataFrame(data)
# teste.head()


In [ ]:
# self.head()

In [ ]:
# self = pd.read_csv("./Titanic.csv")
# condition = self["sex"] == "female"
# conditions = [condition]
# teste = find_delete(self, conditions)
# teste["sex"].value_counts()

In [ ]:
# self = pd.read_csv("./Titanic.csv")
# condition = self["sex"] == "female"
# conditions = [condition]
# replace = ["survived", 1]
# teste = replace.values()
# teste
# teste = find_replace(self, conditions, replace)
# teste4 = find(teste, conditions)
# teste4["survived"].value_counts()

In [ ]:
# self = pd.read_csv("./Titanic.csv")
# teste2 = find(self, [self["sex"] == "female"])
# self.loc[teste2.index, "survived"] = 0
# teste3 = find(self, [self["sex"] == "female"])
# teste3["survived"].value_counts()

In [ ]:
# condition = self["sex"] == "female"
# conditions = [condition]
# replace = [self["survived"], 1]
# teste = replace.values()
# teste
# teste = find_replace(self, conditions, replace)
# teste.head()
# teste1 = find(teste, conditions)
# teste1["survived"].value_counts()
# adw = find(teste, conditions)
# teste.head()
# df["female"].value_counts()

In [ ]:
# teste.head()

In [ ]:
# teste = remove_rows_character(teste, characters=["-", "?", "!", " ", ","], add_blankspace=True)
# teste.head()

In [ ]:
# df1["Product"].dtypes

In [ ]:
# get_nulls_count(df1)

In [ ]:
# df1.info()

In [ ]:
# get_dtype(df1, "Product")

In [ ]:
# df1 = get_best_dtypes(df1, convert=True)

In [ ]:
# df1.info()

In [ ]:
# df1 = remove_rows_with_missing_values(df1)

In [ ]:
# get_col_null_count(df1, ["Purchase Address"])

In [ ]:
Cleaning.split_rows_string(df1, "Purchase Address", ["Address", "City", "State"], delete_col=False)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Address,City,State
0,150502.0,iPhone,1.0,700.00,02/18/19 01:35,"866 Spruce St, Portland, ME 04101",866 Spruce St,"Portland, ME 04101",ME 04101
1,150503.0,AA Batteries (4-pack),1.0,3.84,02/13/19 07:24,"18 13th St, San Francisco, CA 94016",18 13th St,"San Francisco, CA 94016",CA 94016
2,150504.0,27in 4K Gaming Monitor,1.0,389.99,02/18/19 09:46,"52 6th St, New York City, NY 10001",52 6th St,"New York City, NY 10001",NY 10001
3,150505.0,Lightning Charging Cable,1.0,14.95,02/02/19 16:47,"129 Cherry St, Atlanta, GA 30301",129 Cherry St,"Atlanta, GA 30301",GA 30301
4,150506.0,AA Batteries (4-pack),2.0,3.84,02/28/19 20:32,"548 Lincoln St, Seattle, WA 98101",548 Lincoln St,"Seattle, WA 98101",WA 98101
...,...,...,...,...,...,...,...,...,...
372985,295660.0,AAA Batteries (4-pack),2.0,2.99,11/04/19 14:17,"574 4th St, Los Angeles, CA 90001",574 4th St,"Los Angeles, CA 90001",CA 90001
372986,295661.0,USB-C Charging Cable,1.0,11.95,11/23/19 07:22,"359 1st St, Austin, TX 73301",359 1st St,"Austin, TX 73301",TX 73301
372987,295662.0,Lightning Charging Cable,1.0,14.95,11/13/19 16:12,"900 10th St, Boston, MA 02215",900 10th St,"Boston, MA 02215",MA 02215
372988,295663.0,AAA Batteries (4-pack),1.0,2.99,11/17/19 17:08,"592 Sunset St, Boston, MA 02215",592 Sunset St,"Boston, MA 02215",MA 02215
